# RAG part 4

### Integrating RAG into LLM

### Expert Knowledge Worker

A question answering agent that is an expert knowledge worker
To be used by employees of Insurellm, an Insurance Tech company
The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

## 1. Load Data with LangChain

In [5]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

## 2. Split Documents into Chunks

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [7]:
len(chunks)

123

In [8]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: contracts, company, employees, products


## 3. Generate Embeddings Using OpenAI

In [9]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

embeddings = OpenAIEmbeddings()

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 123 documents


## 4. Create or Reset the Chroma Vector Store

In [10]:
# Get one vector and find how many dimensions it has

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


## 5. Integrate Components for Retrieval-Augmented Generation (RAG)


### RAG Workflow Synergy in Conversational Systems

In this step, the workflow demonstrates a synergy between:
- **Auto-regressive text generation** for responses.
- **Auto-encoding-like embeddings** for retrieval.

#### Key Elements Involved
1. **LLM Initialization**: An LLM (e.g., GPT-4o-mini) is initialized for generating conversational responses.
2. **Memory Setup**: Memory is configured to maintain the context of the conversation.
3. **Retriever Configuration**: A retriever is set up to fetch relevant chunks from the vector store during the conversation.
4. **Conversational Chain Integration**: All components are combined into a `ConversationalRetrievalChain` to enable retrieval-augmented responses.

---

### Detailed Explanation

#### Element 2. Memory Setup
The memory in a conversational system serves to:
- **Store Conversation History**: Maintain a record of dialogue exchanges so far.
- **Provide Context for Responses**: Allow the LLM to generate contextually aware and coherent replies.
- **Enhance User Experience**: Enable natural interactions by referencing past messages or decisions.

---

#### Element 3. Working and Effect of the Retriever

Core Functionality:

The retriever uses a vector similarity search process to find and return relevant document chunks. The steps are as follows:
- **Query Encoding**:
  - The retriever receives a query (e.g., user input or LLM-generated question).
  - It converts the query into an embedding using the same embedding model used to create the vector store, ensuring compatibility.
- **Similarity Search**:
  - The vector store contains embeddings for all document chunks stored in high-dimensional space.
  - The retriever calculates the cosine similarity (or another metric) between the query embedding and each stored embedding.
  - The most relevant matches (highest similarity scores) are identified.
- **Document Selection**:
  - The retriever fetches the top-N relevant chunks.
  - These chunks can be filtered or ranked further, depending on application requirements.

#### Element 4. Conversational Chain Integration

- The retrieved chunks are passed to the LLM as part of the context for response generation.
- The LLM integrates the retrieved information into its auto-regressive generation process.
- This process enhances the LLM's ability to provide accurate and context-aware responses, even when the LLM's internal knowledge is limited due to a knowledge cutoff or lack of training data.

--- 


In [11]:
# Creating a new Chat with OpenAI by initializing the 'ChatOpenAI' object.
# Setting the temperature to 0.7 for controlled creativity in responses and specifying the model to use ('MODEL').
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Setting up conversation memory using 'ConversationBufferMemory' to store chat history.
# Using 'memory_key' to name the memory variable and enabling 'return_messages' to retrieve past messages in conversations.
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Setting up the retriever as an abstraction over the VectorStore/a simpler interface to interact with the VectorStore 
# to simplify the process of querying and retrieving documents from the VectorStore during RAG (Retrieval-Augmented Generation).
retriever = vectorstore.as_retriever()

# Combining components into a conversation chain using 'ConversationalRetrievalChain'.
# The chain integrates the GPT-4o-mini language model (llm), the retriever for document access, and the memory for maintaining chat context.
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [12]:
query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

Insurellm is an innovative insurance tech startup founded by Avery Lancaster in 2015, focused on disrupting the insurance industry with its advanced software products. With a workforce of 200 employees and 12 offices across the US, Insurellm offers four main products: Carllm for auto insurance, Homellm for home insurance, Rellm for the reinsurance sector, and Marketllm, a marketplace connecting consumers with insurance providers. The company has rapidly grown to serve over 300 clients worldwide, emphasizing innovation and reliability in the insurance landscape.


In [13]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## 6. Set up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [14]:
# Wrapping in a function - note that history isn't used, as the memory is in the conversation_chain

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [15]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
